In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
import pickle
import math
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model

In [2]:
TRAIN_DIR = r'D:\faces\three_faces\train_dataset'
TRAIN_SAMPLES = 150

VAL_DIR = r'D:\faces\three_faces\validation_dataset'
VAL_SAMPLES = 15

TEST_DIR = r'D:\faces\three_faces\validation_dataset'
CATEGORIES = ['gates', 'musk']
NUM_CLASSES = 2
BATCH_SIZE = 32
IMG_SIZE = 224

In [3]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [4]:
train_generator = train_datagen.flow_from_directory(TRAIN_DIR,
                                                    target_size=(IMG_SIZE, IMG_SIZE),
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True,
                                                    seed=12345,
                                                    class_mode='categorical'
                                                    )

validation_generator = train_datagen.flow_from_directory(VAL_DIR,
                                                    target_size=(IMG_SIZE, IMG_SIZE),
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=False,
                                                    class_mode='categorical'
                                                    )

Found 274 images belonging to 2 classes.
Found 30 images belonging to 2 classes.


In [5]:
def model_maker():
    base_model = MobileNet(include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers[:]:
        layer.trainable = False
    input = Input(shape=(224, 224, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(32, activation="relu")(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(2, activation="softmax")(custom_model)
    return Model(inputs=input, outputs=predictions)

In [6]:
model = model_maker()
model.compile(loss="categorical_crossentropy",
              optimizer=Adam(learning_rate=0.001),
              metrics=['Accuracy']
              )

In [7]:
num_steps = math.ceil(float(TRAIN_SAMPLES) /BATCH_SIZE)

In [8]:
model.fit_generator(train_generator, epochs=10, validation_data= validation_generator,)

C:\Users\user\AppData\Local\Temp\ipykernel_13448\3271922972.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=10, validation_data= validation_generator,)


Epoch 1/10
9/9 [==============================] - 10s 875ms/step - loss: 1.0951 - Accuracy: 0.5255 - val_loss: 0.8302 - val_Accuracy: 0.6333
Epoch 2/10
9/9 [==============================] - 6s 687ms/step - loss: 0.5991 - Accuracy: 0.6788 - val_loss: 0.6638 - val_Accuracy: 0.6667
Epoch 3/10
9/9 [==============================] - 6s 658ms/step - loss: 0.5136 - Accuracy: 0.7482 - val_loss: 0.7062 - val_Accuracy: 0.6667
Epoch 4/10
9/9 [==============================] - 6s 680ms/step - loss: 0.4831 - Accuracy: 0.7701 - val_loss: 0.7202 - val_Accuracy: 0.7000
Epoch 5/10
9/9 [==============================] - 6s 673ms/step - loss: 0.3821 - Accuracy: 0.8540 - val_loss: 0.4946 - val_Accuracy: 0.8333
Epoch 6/10
9/9 [==============================] - 6s 678ms/step - loss: 0.3848 - Accuracy: 0.8394 - val_loss: 0.5255 - val_Accuracy: 0.8000
Epoch 7/10
9/9 [==============================] - 6s 685ms/step - loss: 0.3333 - Accuracy: 0.8723 - val_loss: 0.6175 - val_Accuracy: 0.8000
Epoch 8/10
9/9 [===

In [10]:
model.save(r'D:\faces\model\model.h5')

In [11]:
model = load_model(r'D:\faces\model\model.h5')

In [12]:
TEST_DIR = r'D:\faces\test_data\roflan_ebalo2.jpg'

img = image.load_img(TEST_DIR, target_size=(224, 224))
img_array = image.img_to_array(img)
exp_img_arr = np.expand_dims(img_array, axis=0)
preprocessed_img = preprocess_input(exp_img_arr)


In [13]:
prediction = model.predict(preprocessed_img)
print(prediction)
print(validation_generator.class_indices)

1/1 [==============================] - 1s 634ms/step
[[0.45006695 0.5499331 ]]
{'gates': 0, 'musk': 1}
